In [ ]:
import os

from srai.datasets import HouseSalesInKingCountyDataset

%load_ext dotenv
%dotenv

In [ ]:
hskc_dataset = HouseSalesInKingCountyDataset()
hf_token = os.getenv("HF_TOKEN")

Load default version of dataset

In [ ]:
gdf_train, gfd_test = hskc_dataset.load(hf_token=hf_token)

In [ ]:
gdf_train.head()

Load raw version of dataset

In [ ]:
gdf_all, _ = hskc_dataset.load(hf_token=hf_token, version="all")

In [ ]:
gdf_all.head()